### Importing Datas

In [1]:
from kmer_counting import rna_strand_list_to_matrix, sequence2matrix
import numpy as np 
import pandas as pd
from Bio import SeqIO
import os


In [2]:
circ_rna_path = "dataset/circRNA_fasta"
sequences = [str(i.seq) for i in list(SeqIO.parse(circ_rna_path, "fasta"))]
len(sequences)

32914

In [3]:
converter = sequence2matrix()
matrix = converter.fit(sequences, "vector_array")
matrix.shape

AttributeError: 'list' object has no attribute 'shape'

In [7]:
np.load("dataset/vector_array.npy", allow_pickle=True)

array([{'AAAA': 1, 'AAAG': 3, 'AAAC': 2, 'AAAU': 1, 'AAGA': 5, 'AAGG': 2, 'AAGC': 1, 'AAGU': 2, 'AACA': 3, 'AACG': 1, 'AACC': 3, 'AACU': 0, 'AAUA': 0, 'AAUG': 1, 'AAUC': 0, 'AAUU': 1, 'AGAA': 4, 'AGAG': 2, 'AGAC': 4, 'AGAU': 3, 'AGGA': 3, 'AGGG': 0, 'AGGC': 4, 'AGGU': 3, 'AGCA': 3, 'AGCG': 6, 'AGCC': 6, 'AGCU': 3, 'AGUA': 1, 'AGUG': 1, 'AGUC': 1, 'AGUU': 1, 'ACAA': 4, 'ACAG': 3, 'ACAC': 0, 'ACAU': 4, 'ACGA': 0, 'ACGG': 3, 'ACGC': 2, 'ACGU': 3, 'ACCA': 2, 'ACCG': 1, 'ACCC': 3, 'ACCU': 0, 'ACUA': 0, 'ACUG': 2, 'ACUC': 2, 'ACUU': 2, 'AUAA': 0, 'AUAG': 0, 'AUAC': 0, 'AUAU': 0, 'AUGA': 5, 'AUGG': 1, 'AUGC': 2, 'AUGU': 2, 'AUCA': 1, 'AUCG': 1, 'AUCC': 3, 'AUCU': 3, 'AUUA': 0, 'AUUG': 1, 'AUUC': 2, 'AUUU': 0, 'GAAA': 4, 'GAAG': 3, 'GAAC': 3, 'GAAU': 0, 'GAGA': 2, 'GAGG': 4, 'GAGC': 7, 'GAGU': 1, 'GACA': 3, 'GACG': 3, 'GACC': 1, 'GACU': 3, 'GAUA': 0, 'GAUG': 3, 'GAUC': 4, 'GAUU': 0, 'GGAA': 2, 'GGAG': 8, 'GGAC': 1, 'GGAU': 1, 'GGGA': 2, 'GGGG': 0, 'GGGC': 2, 'GGGU': 2, 'GGCA': 0, 'GGCG': 1, 'G